# Tutorial: Using Optuna's Bayesian optimization to tune hyperparameters 


It is highly recommended that the Bayesian optimization routine is executed in an environment **with access to CUDA and/or OpenMP**, as it greatly accelerates the entire process.

It assumes that pysdg is already installed in a Conda environment, the environment has been activated from the shell, and this notebook is being run within that activated environment. For detailed instructions, please refer to the "pysdg" documentation.

The following cell sets the working directory to the location of this notebook. It is assumed that all files accessed by this notebook are stored in the same directory.

### Import

First we import the necessary packages and apply the proper settings for prettier display of both Pandas data frames and Python dictionaries. The last line below imports the  Generator class from `pysdg` synth module. 

In [2]:
from pysdg.gen import Generator
from pysdg.optimize import BayesianOptimizationRoutine

### Choose the Generator and Load the Training Data

We use CTGAN to demonstrate how CTGAN can be applied in synthetic data generation.

In [4]:
gen = Generator(gen_name="synthcity/ctgan")
real=gen.load("./raw_data.csv", "./raw_info.json")

2025-05-12 15:28:18,534 - pysdg - INFO - 3394 - generate.py:122 - **************Started logging the generator: synthcity/ctgan, num_cores= None.**************
2025-05-12 15:28:18,546 - pysdg - INFO - 3394 - generate.py:347 - Checking the input metadata for any conflict in variable indexes - Passed.
2025-05-12 15:28:19,294 - pysdg - INFO - 3394 - generate.py:461 - The dataset ['tutorial_data'] is loaded into the generator synthcity_ctgan


In [5]:
# Define you own evaluation function
def my_eval_function(gen: Generator):
    real_data = gen.enc_real
    synth_data = gen.enc_synths[0] # we'll be assuming that we're generating only one dataset and we compare the encoded datasets, for simplicity
    n_mismatches  = (real_data != synth_data).sum().sum()
    return n_mismatches



In [6]:
bayes_opt = BayesianOptimizationRoutine(
                                        gen=gen,
                                        eval_function=my_eval_function,
                                        objective="minimize",
                                        n_trials=1, # to make it finish faster
                                        study_name="mismatches_study",
                                        dump_csv=False, # dumping csv will only happen at the end of the optimization
                                        dump_sqlite=False # dumping sql happens after each trial
                                        )

2025-05-12 15:28:24,818 - pysdg - INFO - 3394 - generate.py:942 - Started training using synthcity_ctgan...
[2025-05-12T15:28:24.844402-0400][3394][CRITICAL] module disabled: /home/samer/miniconda3/envs/pysdgdev/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
2025-05-12 15:28:26,578 - pysdg - INFO - 3394 - generate.py:946 - No of Iterations=15, Batch Size=128
INFO:pysdg:No of Iterations=15, Batch Size=128
100%|██████████| 15/15 [00:19<00:00,  1.28s/it]
2025-05-12 15:28:52,325 - pysdg - INFO - 3394 - generate.py:954 - Completed training using synthcity_ctgan.
INFO:pysdg:Completed training using synthcity_ctgan.
2025-05-12 15:28:52,500 - pysdg - INFO - 3394 - generate.py:981 - Generating synth no. 0 of size (10000, 12) -- Completed!
INFO:pysdg:Generating synth no. 0 of size (10000, 12) -- Completed!
2025-05-12 15:28:52,563 - pysdg - INFO - 3394 - generate.py:942 - Started training using synthcity_ctgan...
INFO:pysdg:Started training using synthcity_ctgan...
[2025-

In [7]:
bayes_opt.best_gen.gen(num_rows=len(real), num_synths=1)
synths=bayes_opt.best_gen.unload()
synths[0]


2025-05-12 15:33:44,668 - pysdg - INFO - 3394 - generate.py:981 - Generating synth no. 0 of size (10000, 12) -- Completed!
INFO:pysdg:Generating synth no. 0 of size (10000, 12) -- Completed!


,outc_cod_0,event_dt,wt,wt_cod,age,age_cod,drugname_0,indi_pt_0,sex
0,NaN,NaT,NaN,NaN,<NA>,NaN,CLENPIQ,Acne conglobata,NaN
1,NaN,NaT,NaN,NaN,<NA>,YR,DOXYCYCLINE.,Rectosigmoid cancer,F
2,NaN,NaT,NaN,NaN,54,NaN,TACROLIMUS,Psoriasis,NaN
3,NaN,NaT,NaN,NaN,<NA>,YR,ABIRATERONE,Acromegaly,F
4,NaN,NaT,NaN,NaN,<NA>,YR,Lurasidone,Skin wrinkling,F
...,...,...,...,...,...,...,...,...,...
9995,NaN,2018-07-25,NaN,NaN,<NA>,YR,METHYLPREDNISOLONE.,Inflammation,F
9996,OT,2019-01-05,NaN,KG,108,NaN,EXELON,Aneurysm repair,M
9997,LT,2019-06-06,NaN,KG,<NA>,YR,EFAVIRENZ.,Schizophrenia,M
9998,NaN,NaT,NaN,NaN,<NA>,NaN,DABRAFENIB,Abdominal pain,F
